<a href="https://colab.research.google.com/github/DivyaDharshini2293/Big-Data-Analytics/blob/main/BDA_MiniProject_Network_Anomaly_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Anomaly Detection

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=bfd66726c7821bf9d616b644065f9177bf8d4d169e1be758cb3af9fd0f64f04a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark.sql.functions as funcs
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import *
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder\
.master("local[4]")\
.appName("ReadFromCsv")\
.config("spark.driver.memory","3g")\
.config("spark.executor.memory", "4g")\
.getOrCreate()

In [ ]:
data = spark.read \
.format("csv")\
.option("header", True)\
.option("sep", ",")\
.option("inferSchema", "True")\
.load("/content/TrainDf.csv")

In [ ]:
data.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- srv_count: integer (nullable = true)
 |-- serror_rate: double (nullable = true)
 |-- srv_serror_rate: double (nul

In [ ]:
data.show()

+--------+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_same_srv_rate|dst_

In [ ]:
data.describe().show()

+-------+------------------+-----------------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-----------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|summary|          duration|        src_bytes|         dst_bytes|                land|      wrong_fragment|              urgent|                hot|   num_failed_logins|          logged_in| 

In [ ]:
print("Shape:", (data.count(), len(data.columns)))

Shape: (125973, 39)


In [ ]:
from pyspark.sql.functions import col,isnan, when, count
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]
   ).show()

+--------+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+------+
|duration|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_same_srv_rate|dst_h

In [ ]:
data.groupBy("status").count().show()

+-------+-----+
| status|count|
+-------+-----+
| normal|67343|
|anomaly|58630|
+-------+-----+



In [ ]:
data1=data.orderBy(data["count"].asc())
data1.show(n=20,truncate=False)

+--------+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_same_srv_rate|dst_

In [ ]:
numeric_columns = [column[0] for column in data.dtypes if column[1]=='int']
numeric_columns

['duration',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'dst_host_count',
 'dst_host_srv_count']

#Detecting Outliers using InterQuartile Range

 Outliers often represent unusual or rare observations in a dataset. Detecting these anomalies can be valuable for various applications, such as fraud detection, network security, and fault detection in industrial processes.

In [ ]:
def find_outliers(df):
    numeric_columns = [column[0] for column in df.dtypes if column[1]=='int']
    for column in numeric_columns:

        less_Q1 = 'less_Q1_{}'.format(column)
        more_Q3 = 'more_Q3_{}'.format(column)
        Q1 = 'Q1_{}'.format(column)
        Q3 = 'Q3_{}'.format(column)
        Q1 = df.approxQuantile(column,[0.25],relativeError=0)
        Q3 = df.approxQuantile(column,[0.75],relativeError=0)
        IQR = Q3[0] - Q1[0]
        less_Q1 =  Q1[0] - 1.5*IQR
        more_Q3 =  Q3[0] + 1.5*IQR
        isOutlierCol = 'is_outlier_{}'.format(column)
        df = df.withColumn(isOutlierCol,f.when((df[column] > more_Q3) | (df[column] < less_Q1), 1).otherwise(0))


    selected_columns = [column for column in df.columns if column.startswith("is_outlier")]
    df = df.withColumn('total_outliers',sum(df[column] for column in selected_columns))
    df = df.drop(*[column for column in df.columns if column.startswith("is_outlier")])
    return df

In [ ]:
from pyspark.sql import functions as f
new_df = find_outliers(data)
new_df.show()

+--------+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+--------------+
|duration|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_sam

In [ ]:
df_no_outliers = new_df.filter(new_df['total_Outliers']<=1)
df_no_outliers = df_no_outliers.select(*data.columns)

df_no_outliers.show()

+--------+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_same_srv_rate|dst_

In [ ]:
df_no_outliers.count()

114634

In [ ]:
data_with_outliers = new_df.filter(new_df['total_Outliers']>=2)
data_with_outliers.show()

+--------+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+--------------+
|duration|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_sam

In [ ]:
data_with_outliers.count()

11339

In [ ]:
numeric_columns

['duration',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'dst_host_count',
 'dst_host_srv_count']

In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml import Pipeline


In [ ]:
feature_cols = df_no_outliers.columns[:-1]

In [ ]:
feature_cols

['duration',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate']

In [ ]:
label_indexer = StringIndexer(inputCol = "status", outputCol = "label")
label_indexer

StringIndexer_aaf02705985b

In [ ]:
assembler = VectorAssembler(inputCols = feature_cols, outputCol = 'features')

In [ ]:
pipe = Pipeline(stages=[assembler, label_indexer])

Pipeline_b91bc7ea7449

In [ ]:
pipe_model = pipe.fit(data)

In [ ]:
data = pipe_model.transform(data)
data = data.select("features","label")
data.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(38,[1,19,20,25,2...|  0.0|
|(38,[1,19,20,25,2...|  0.0|
|(38,[19,20,21,22,...|  1.0|
|(38,[1,2,8,19,20,...|  0.0|
|(38,[1,2,8,19,20,...|  0.0|
|(38,[19,20,23,24,...|  1.0|
|(38,[19,20,21,22,...|  1.0|
|(38,[19,20,21,22,...|  1.0|
|(38,[19,20,21,22,...|  1.0|
|(38,[19,20,21,22,...|  1.0|
|(38,[19,20,23,24,...|  1.0|
|(38,[19,20,21,22,...|  1.0|
|(38,[1,2,8,19,20,...|  0.0|
|(38,[1,8,19,20,25...|  1.0|
|(38,[19,20,21,22,...|  1.0|
|(38,[19,20,21,22,...|  1.0|
|(38,[1,2,8,19,20,...|  0.0|
|(38,[1,19,20,25,2...|  1.0|
|(38,[1,2,8,19,20,...|  0.0|
|(38,[1,2,8,19,20,...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [ ]:
train, test = data.randomSplit([0.70, 0.30])


# Decision Tree Algorithm

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
model = dt.fit(train)
predict = model.transform(test)
predict.toPandas().head()

,features,label,rawPrediction,probability,prediction
0,"(7805.0, 1828.0, 148722.0, 0.0, 0.0, 0.0, 1.0,...",0.0,"[626.0, 44.0]","[0.9343283582089552, 0.06567164179104477]",0.0
1,"(169.0, 1567.0, 2857.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[626.0, 44.0]","[0.9343283582089552, 0.06567164179104477]",0.0
2,"(176.0, 1559.0, 2732.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[626.0, 44.0]","[0.9343283582089552, 0.06567164179104477]",0.0
3,"(454.0, 492.0, 14334.0, 0.0, 0.0, 0.0, 2.0, 0....",0.0,"[626.0, 44.0]","[0.9343283582089552, 0.06567164179104477]",0.0
4,"(454.0, 492.0, 14334.0, 0.0, 0.0, 0.0, 2.0, 0....",0.0,"[626.0, 44.0]","[0.9343283582089552, 0.06567164179104477]",0.0


In [ ]:
predict.select("prediction", "label")\
.groupBy("prediction", "label").count()\
.orderBy("prediction", "label", ascending=True).withColumn("status",
funcs.when(funcs.col("label").isin(1), "Anomaly")\
.otherwise("Normal")).toPandas().head()

,prediction,label,count,status
0,0.0,0.0,20177,Normal
1,0.0,1.0,257,Anomaly
2,1.0,0.0,94,Normal
3,1.0,1.0,17570,Anomaly


In [ ]:
predict.groupBy(["label","prediction"]).count().toPandas().head()

,label,prediction,count
0,1.0,1.0,17570
1,0.0,1.0,94
2,1.0,0.0,257
3,0.0,0.0,20177


In [ ]:
evaluatordt = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
dt = evaluatordt.evaluate(predict)

print("--- Decision Tree --- ")
print("Accuracy Rate =", round(dt,4))
print("  Error  Rate = %g " % round((1.0 - dt),4))

--- Decision Tree --- 
Accuracy Rate = 0.9908
  Error  Rate = 0.0092 


In [ ]:
predictionAndLabel = predict.select("prediction", "label").rdd

from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(predictionAndLabel)
cm = metrics.confusionMatrix()
rows = cm.toArray().tolist()

confusion_matrix = spark.createDataFrame(rows,["normal","anomaly"])
confusion_matrix.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------+-------+
| normal|anomaly|
+-------+-------+
|20177.0|   94.0|
|  257.0|17570.0|
+-------+-------+



In [ ]:
predict.withColumn("A", funcs.struct("prediction","label")).crosstab("prediction","label").show()

+----------------+-----+-----+
|prediction_label|  0.0|  1.0|
+----------------+-----+-----+
|             1.0|   94|17570|
|             0.0|20177|  257|
+----------------+-----+-----+



#Random Forest Algorithm

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)
modelrf = rf.fit(train)
predictionrf = modelrf.transform(test)
predictionrf.toPandas().head(3)

,features,label,rawPrediction,probability,prediction
0,"(7805.0, 1828.0, 148722.0, 0.0, 0.0, 0.0, 1.0,...",0.0,"[8.926795914022238, 1.0732040859777625]","[0.8926795914022237, 0.10732040859777625]",0.0
1,"(169.0, 1567.0, 2857.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[8.445503726650804, 1.554496273349197]","[0.8445503726650804, 0.15544962733491968]",0.0
2,"(176.0, 1559.0, 2732.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[7.546855172540107, 2.4531448274598926]","[0.7546855172540108, 0.24531448274598927]",0.0


In [ ]:
evaluatorrf = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
rf = evaluatorrf.evaluate(predictionrf)

print("--- Random Forest Tree --- ")
print("Accuracy Rate =", round(rf,4))
print("  Error  Rate = %g " % round((1.0 - rf),4))

--- Random Forest Tree --- 
Accuracy Rate = 0.9833
  Error  Rate = 0.0167 


In [ ]:
predictionAndLabel = predictionrf.select("prediction", "label").rdd

from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(predictionAndLabel)
cm = metrics.confusionMatrix()
rows = cm.toArray().tolist()

confusion_matrix = spark.createDataFrame(rows,["normal","anomaly"])
confusion_matrix.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------+-------+
| normal|anomaly|
+-------+-------+
|20210.0|   61.0|
|  576.0|17251.0|
+-------+-------+



In [ ]:
predictionAndLabels = predictionrf.select("prediction", "label").rdd
metrics = MulticlassMetrics(predictionAndLabels)
metrics.confusionMatrix()

DenseMatrix(2, 2, [20210.0, 576.0, 61.0, 17251.0], 0)

In [ ]:
predictionrf.withColumn("A", funcs.struct("prediction","label")).crosstab("prediction","label").show()

+----------------+-----+-----+
|prediction_label|  0.0|  1.0|
+----------------+-----+-----+
|             1.0|   61|17251|
|             0.0|20210|  576|
+----------------+-----+-----+



In [ ]:
predictionrfevaluator = MulticlassClassificationEvaluator(predictionCol="prediction", \
                    labelCol="label",metricName="accuracy")
predictionrfevaluator.evaluate(predictionrf)

predictionrf.groupBy("label","prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|17251|
|  0.0|       1.0|   61|
|  1.0|       0.0|  576|
|  0.0|       0.0|20210|
+-----+----------+-----+



#Naive Bayes Algorithm

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

modelnb = nb.fit(train)
predictionnb = modelnb.transform(test)
predictionnb.toPandas().head(3)

,features,label,rawPrediction,probability,prediction
0,"(7805.0, 1828.0, 148722.0, 0.0, 0.0, 0.0, 1.0,...",0.0,"[-254174.55726154102, -699080.9691586777]","[1.0, 0.0]",0.0
1,"(169.0, 1567.0, 2857.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[-5597.919087033793, -13767.568325344111]","[1.0, 0.0]",0.0
2,"(176.0, 1559.0, 2732.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[-5505.140191341054, -13345.688982123]","[1.0, 0.0]",0.0


In [ ]:
evaluatornb = MulticlassClassificationEvaluator(labelCol="label",
                                              predictionCol="prediction",
                                              metricName="accuracy")
nb = evaluatornb.evaluate(predictionnb)

print("--- Naive Bayes --- ")
print("Accuracy Rate =", round(nb,4))
print("  Error  Rate = %g " % round((1.0 - nb),4))

--- Naive Bayes --- 
Accuracy Rate = 0.5148
  Error  Rate = 0.4852 


#Gradient Boost Tree

In [ ]:
from pyspark.ml.classification import GBTClassifier

In [ ]:
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

modelgbt = gbt.fit(train)
predictiongbt = modelgbt.transform(test)
predictiongbt.toPandas().head(3)

,features,label,rawPrediction,probability,prediction
0,"(7805.0, 1828.0, 148722.0, 0.0, 0.0, 0.0, 1.0,...",0.0,"[0.715082628162642, -0.715082628162642]","[0.8069270633104179, 0.19307293668958214]",0.0
1,"(169.0, 1567.0, 2857.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[-0.410659872105877, 0.410659872105877]","[0.30548358666165787, 0.6945164133383421]",1.0
2,"(176.0, 1559.0, 2732.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[-0.4590605408355746, 0.4590605408355746]","[0.2853408914954979, 0.7146591085045021]",1.0


In [ ]:
evaluatorgbt = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
gbt = evaluatorgbt.evaluate(predictiongbt)

print("--- Gradient Boost Tree --- ")
print("Accuracy Rate =", round(gbt,4))
print("  Error  Rate = %g " % round((1.0 - gbt),4))

--- Gradient Boost Tree --- 
Accuracy Rate = 0.9914
  Error  Rate = 0.0086 


# Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(regParam=0.01)
modellr = lr.fit(train)
predictionlr = modellr.transform(test)
predictionlr.toPandas().head(3)

,features,label,rawPrediction,probability,prediction
0,"(7805.0, 1828.0, 148722.0, 0.0, 0.0, 0.0, 1.0,...",0.0,"[4.845622865108057, -4.845622865108057]","[0.9921986215265393, 0.0078013784734607405]",0.0
1,"(169.0, 1567.0, 2857.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[-1.338934239940748, 1.338934239940748]","[0.20768537704588025, 0.7923146229541198]",1.0
2,"(176.0, 1559.0, 2732.0, 0.0, 0.0, 0.0, 3.0, 0....",1.0,"[0.7174887591107003, -0.7174887591107003]","[0.6720537851912197, 0.3279462148087803]",0.0


In [ ]:
evaluatorlr = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
lr = evaluatorlr.evaluate(predictionlr)

In [ ]:
print("--- Logistic Regression --- ")
print("Accuracy Rate =", round(lr,4))
print("  Error  Rate = %g " % round((1.0 - lr),4))

--- Logistic Regression --- 
Accuracy Rate = 0.9467
  Error  Rate = 0.0533 


######Comparison of Accucary Rate of Algorithms


In [ ]:
print("Gradient Boost Tree Accuracy =", round(gbt,5))
print("      Decision Tree Accuracy =", round(dt,5))
print(" Random Forest Tree Accuracy =", round(rf,5))
print("Logistic Regression Accuracy =", round(lr,5))
print("        Naive Bayes Accuracy =", round(nb,5))

Gradient Boost Tree Accuracy = 0.99136
      Decision Tree Accuracy = 0.99079
 Random Forest Tree Accuracy = 0.98328
Logistic Regression Accuracy = 0.94672
        Naive Bayes Accuracy = 0.51483
